In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, cross

from backtesting.test import SMA, GOOG

In [3]:
import pandas as pd
import numpy as np

In [4]:
sp = pd.read_csv(r'https://www.dropbox.com/s/lzw7e7yxbuqtokj/sp_d_s.csv?dl=1', index_col=0)

In [5]:
sp

,hour,spread,bts,an_frate,an_brate,time_to_exp
date,,,,,,
2019-03-05,15,-0.002979,3826.589286,-3.951480,-0.628404,27.0
2019-03-05,16,-0.003039,3849.510638,-4.028253,-1.282234,27.0
2019-03-05,17,-0.002882,3833.534091,-3.825478,-0.550230,27.0
2019-03-05,18,-0.002934,3837.073529,-3.893304,-0.978581,27.0
2019-03-05,19,-0.002845,3843.950000,-3.775014,-0.641528,27.0
...,...,...,...,...,...,...
2020-03-16,3,0.050160,5277.630000,2.713106,0.859405,111.0
2020-03-16,4,0.050405,5253.616667,2.786709,0.714465,111.0
2020-03-16,5,0.050306,5224.116667,2.756983,0.180272,111.0


In [7]:
def min_critical(ar, day):
    flag = 0
    for i in range(1, 6):
        if(ar.iloc[day-i]['an_brate'] > ar.iloc[day]['an_brate'] and 
                        ar.iloc[day+i]['an_brate'] > ar.iloc[day]['an_brate']):
            flag = 1
        else:
            flag = 0
            break
    return flag

def max_critical(ar, day):
    flag = 0
    for i in range(1, 6):
        if(ar.iloc[day-i]['an_brate'] < ar.iloc[day]['an_brate'] and 
                        ar.iloc[day+i]['an_brate'] < ar.iloc[day]['an_brate']):
            flag = 1
        else:
            flag = 0
            break
    return flag    

In [8]:
High = []
Low = []
Open = []
Close = []
for i in sp.index.unique():
    temp = sp.loc[i]['spread']
    min_ = temp.iloc[0]
    max_ = temp.iloc[0]
    for j in (1, temp.size-1):
        if(min_ > temp.iloc[j]): min_ = temp.iloc[j]
        if(max_ < temp.iloc[j]): max_ = temp.iloc[j]
    High.append(max_)
    Low.append(min_)
    Open.append(temp.iloc[0])
    Close.append(temp.iloc[temp.size-1])
    
ar = []
for i in sp.index.unique():
    ar.append(pd.Timestamp(i))

test = pd.DataFrame(data = {'Open':Open, 'High':High, 'Low':Low, 
                            'Close':Close, '':ar})

test  = test.set_index('')

buy = [] # perp - xbtu
sell = []
for i in range(1, sp.shape[0]):
    if(i > 5 and i < sp.shape[0]-5 and sp.iloc[i]['an_brate'] < 1 and (min_critical(sp, i))):
        buy.append(i)#sp.iloc[i].name) 
        
    if(i > 5 and i < sp.shape[0]-5 and sp.iloc[i]['an_brate'] > 8 and (max_critical(sp, i))):
        sell.append(i)#sp.iloc[i].name) 

In [9]:
test1 = test[test.index < '2019-12-20']
test2 = test[test.index >= '2019-12-20']

In [10]:
ar = []
for i in sp.index.unique():
    ar.append(pd.Timestamp(i))

test = pd.DataFrame(data = {'Open':Open, 'High':High, 'Low':Low, 
                            'Close':Close, '':ar})

test  = test.set_index('')

In [11]:
test1 = test[test.index < '2019-12-20']
test2 = test[test.index >= '2019-12-20']

In [12]:
test[70:test.shape[0]]

,Open,High,Low,Close
,,,,
2019-05-14,0.002533,0.002767,0.001432,0.001432
2019-05-15,0.001293,0.002017,0.001293,0.002017
2019-05-16,0.002324,0.002324,0.000753,0.000753
2019-05-17,0.000586,0.004679,0.000543,0.004679
2019-05-18,0.004419,0.004419,0.001051,0.001051
...,...,...,...,...
2020-03-12,0.062230,0.062560,0.049766,0.049766
2020-03-13,0.051265,0.051265,0.041296,0.042137
2020-03-14,0.042407,0.043334,0.042407,0.043334


In [13]:
class SmaCross(Strategy):
    n1 = 7#26
    n2 = 14#58

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    def next(self):
        if (cross(self.sma1, self.sma2)):
            self.buy()
        elif (cross(self.sma2, self.sma1)):
            self.sell()


bt = Backtest(test[70:test.shape[0]], SmaCross, cash=10, commission=0.00075)

output = bt.run()
bt.plot()

In [14]:
stats = bt.optimize(n1=range(2, 30, 1),
                    n2=range(2, 70, 1),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2)

C:\Users\molot\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Searching best of 1498 configurations.
  after removing the cwd from sys.path.


In [15]:
stats._strategy

<Strategy SmaCross(n1=14,n2=24)>

In [18]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 30
    
    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.data.index
    def next(self):
        if (cross(self.sma2, self.sma1)):
            self.buy()
        elif (cross(self.sma1, self.sma2)):
            self.sell()


bt = Backtest(test1[70:test1.shape[0]], SmaCross, cash=100, commission=0.00075)

output = bt.run()
bt.plot()

In [41]:
stats = bt.optimize(n1=range(2, 7, 1),
                    n2=range(2, 20, 1),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2)

In [42]:
stats._strategy

<Strategy SmaCross(n1=2,n2=3)>

In [17]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 3

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.data.index
    def next(self):
        if (cross(self.sma2, self.sma1)):
            self.buy()
        elif (cross(self.sma1, self.sma2)):
            self.sell()


bt = Backtest(test2, SmaCross, cash=100, commission=0.00075)

output = bt.run()
bt.plot()

In [ ]:
stats = bt.optimize(n1=range(2, 7, 1),
                    n2=range(2, 20, 1),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2)

In [ ]:
stats._strategy